<a href="https://colab.research.google.com/github/Reape4er/image_processing/blob/main/%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%964.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#запускать в десктопном IDE
import cv2
import numpy as np
# import matplotlib.pyplot as plt

def find_paper_contour(image):
    """Ищет контур прямоугольного листа на изображении."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 50, 150)

    # Поиск контуров
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Аппроксимация контура
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Если контур имеет 4 вершины, возможно, это лист
        if len(approx) == 4:
            return approx

    return None

def rub_detection_sorted(image, blur=3, dp=1.2, minDist=100, param1=50, param2=150, minRadius=50, maxRadius=250, paper_size_mm=(297,210)):
    ''' находит и подсчитывает монеты '''
    # Найти контур листа бумаги
    paper_contour = find_paper_contour(image)

    if paper_contour is None:
        print("лист бумаги не найден.")
        return

    # находим bounding box для листа и определяем масштаб
    paper_rect = cv2.boundingRect(paper_contour)
    paper_width_px = paper_rect[2]  # Ширина прямоугольника в пикселях
    scale_px_per_mm = paper_width_px / paper_size_mm[0] # подсчет масштаба пикселя на миллиметр через ширину

    # в оттенки серого
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # размытие чтобы убрать шум
    gray_blurred = cv2.GaussianBlur(gray, (blur, blur), 0)
    # gray_blurred = cv2.medianBlur(gray, blur)

    # Обнаружение кругов
    circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, dp, minDist,
                               param1=param1, param2=param2,
                               minRadius=minRadius, maxRadius=maxRadius)

    if circles is not None:
        # Округление координат центров и радиусов до целых чисел
        circles = np.uint16(np.around(circles))

        # Диаметры монет в миллиметрах
        coin_sizes_mm = {
            "1 рубль": [1, 20.50],
            "2 рубля": [2, 23.00],
            "5 рублей": [5, 25.00],
            # "10 рублей": 22.00
        }

        img_circles = image.copy()

        total = 0
        for circle in circles[0, :]:
            center = (circle[0], circle[1])  # Координаты центра
            radius_px = circle[2]  # Радиус в пикселях

            # Перевод радиуса в миллиметры
            diameter_mm = (radius_px * 2) / scale_px_per_mm

            # Определение монеты по диаметру
            detected_coin = None
            for coin, params in coin_sizes_mm.items():
                if abs(diameter_mm - params[1]) <= 1:  # Допустимая погрешность 1 мм
                    detected_coin = coin
                    total+= params[0]
                    break

            # Рисование контура круга
            cv2.circle(img_circles, center, radius_px, (0, 255, 0), 1)

            # Рисование центра круга
            cv2.circle(img_circles, center, 2, (0, 0, 255), 3)

            # Подпись монеты
            label = detected_coin if detected_coin else f"Неизвестно ({diameter_mm:.1f} мм)"

            cv2.putText(img_circles, label, (center[0] - 50, center[1] - 10),
                        cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 0, 0), 2)

            cv2.drawContours(img_circles, [paper_contour], -1, (255, 0, 0), 2)
        cv2.putText(img_circles, str(total), (50,250),
                    cv2.FONT_HERSHEY_COMPLEX, 0.8, (0,0,255), 2)
        return img_circles

    else:
        print("Круги не найдены")
        return image

# Открытие видеопотока с камеры
cap = cv2.VideoCapture(0)
# Создание окон
cv2.namedWindow('Stream')

# Создание трекбаров для параметров
cv2.createTrackbar('Blur', 'Stream', 5, 20, lambda x: None)
cv2.createTrackbar('dp', 'Stream', 10, 30, lambda x: None)
cv2.createTrackbar('MinDist', 'Stream', 20, 100, lambda x: None)
cv2.createTrackbar('Param1', 'Stream', 150, 300, lambda x: None)
cv2.createTrackbar('Param2', 'Stream', 30, 100, lambda x: None)
cv2.createTrackbar('MinRadius', 'Stream', 0, 100, lambda x: None)
cv2.createTrackbar('MaxRadius', 'Stream', 0, 100, lambda x: None)
while True:
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    try:
        # Чтение значений из трекбаров
        blur = cv2.getTrackbarPos('Blur', 'Stream')
        dp = cv2.getTrackbarPos('dp', 'Stream') / 10.0  # Приводим к нужному диапазону
        minDist = cv2.getTrackbarPos('MinDist', 'Stream')
        param1 = cv2.getTrackbarPos('Param1', 'Stream')
        param2 = cv2.getTrackbarPos('Param2', 'Stream')
        minRadius = cv2.getTrackbarPos('MinRadius', 'Stream')
        maxRadius = cv2.getTrackbarPos('MaxRadius', 'Stream')

        detected_image = rub_detection_sorted(frame, blur=blur, dp=dp, minDist=minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)

        cv2.imshow('Stream', detected_image)
    except:
        cv2.imshow('Stream', frame)
    # Выход при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()
